# [SortedSet](https://github.com/tatyam-prime/SortedSet)を使えるようになろう

参考
- [Python で std::set の代替物を作った](https://qiita.com/tatyam/items/492c70ac4c955c055602)

In [56]:
# https://github.com/tatyam-prime/SortedSet/blob/main/SortedSet.py
import math
from bisect import bisect_left, bisect_right
from typing import Generic, Iterable, Iterator, TypeVar, Union, List
T = TypeVar('T')

class SortedSet(Generic[T]):
    BUCKET_RATIO = 50
    REBUILD_RATIO = 170

    def _build(self, a=None) -> None:
        "Evenly divide `a` into buckets."
        if a is None: a = list(self)
        size = self.size = len(a)
        bucket_size = int(math.ceil(math.sqrt(size / self.BUCKET_RATIO)))
        self.a = [a[size * i // bucket_size : size * (i + 1) // bucket_size] for i in range(bucket_size)]
    
    def __init__(self, a: Iterable[T] = []) -> None:
        "Make a new SortedSet from iterable. / O(N) if sorted and unique / O(N log N)"
        a = list(a)
        if not all(a[i] < a[i + 1] for i in range(len(a) - 1)):
            a = sorted(set(a))
        self._build(a)

    def __iter__(self) -> Iterator[T]:
        for i in self.a:
            for j in i: yield j

    def __reversed__(self) -> Iterator[T]:
        for i in reversed(self.a):
            for j in reversed(i): yield j
    
    def __len__(self) -> int:
        return self.size
    
    def __repr__(self) -> str:
        return "SortedSet" + str(self.a)
    
    def __str__(self) -> str:
        s = str(list(self))
        return "{" + s[1 : len(s) - 1] + "}"

    def _find_bucket(self, x: T) -> List[T]:
        "Find the bucket which should contain x. self must not be empty."
        for a in self.a:
            if x <= a[-1]: return a
        return a

    def __contains__(self, x: T) -> bool:
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        return i != len(a) and a[i] == x

    def add(self, x: T) -> bool:
        "Add an element and return True if added. / O(√N)"
        if self.size == 0:
            self.a = [[x]]
            self.size = 1
            return True
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i != len(a) and a[i] == x: return False
        a.insert(i, x)
        self.size += 1
        if len(a) > len(self.a) * self.REBUILD_RATIO:
            self._build()
        return True

    def discard(self, x: T) -> bool:
        "Remove an element and return True if removed. / O(√N)"
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i == len(a) or a[i] != x: return False
        a.pop(i)
        self.size -= 1
        if len(a) == 0: self._build()
        return True
    
    def lt(self, x: T) -> Union[T, None]:
        "Find the largest element < x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] < x:
                return a[bisect_left(a, x) - 1]

    def le(self, x: T) -> Union[T, None]:
        "Find the largest element <= x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] <= x:
                return a[bisect_right(a, x) - 1]

    def gt(self, x: T) -> Union[T, None]:
        "Find the smallest element > x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] > x:
                return a[bisect_right(a, x)]

    def ge(self, x: T) -> Union[T, None]:
        "Find the smallest element >= x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] >= x:
                return a[bisect_left(a, x)]
    
    def __getitem__(self, x: int) -> T:
        "Return the x-th element, or IndexError if it doesn't exist."
        if x < 0: x += self.size
        if x < 0: raise IndexError
        for a in self.a:
            if x < len(a): return a[x]
            x -= len(a)
        raise IndexError
    
    def index(self, x: T) -> int:
        "Count the number of elements < x."
        ans = 0
        for a in self.a:
            if a[-1] >= x:
                return ans + bisect_left(a, x)
            ans += len(a)
        return ans

    def index_right(self, x: T) -> int:
        "Count the number of elements <= x."
        ans = 0
        for a in self.a:
            if a[-1] > x:
                return ans + bisect_right(a, x)
            ans += len(a)
        return ans


## 使ってみる

In [57]:
MAX = 1e5
N = 300

In [58]:
from random import randint
data = [randint(0, MAX) for _ in range(N)]

In [59]:
ss = SortedSet(data)

In [60]:
for row in ss.a:
    print(row)

[854, 935, 953, 1410, 2306, 2360, 2442, 3169, 3451, 4162, 4394, 4753, 5030, 5403, 5454, 6380, 6487, 7831, 8452, 8594, 8926, 10074, 11793, 12153, 12169, 12555, 12645, 12688, 12813, 12831, 13343, 13368, 13690, 13756, 13820, 13894, 14106, 14203, 14458, 14607, 14877, 15845, 15943, 16012, 16069, 16649, 16916, 17349, 17555, 17604, 17678, 17843, 18276, 19126, 19545, 19980, 20088, 20659, 21338, 21772, 22257, 23370, 23475, 23849, 23968, 24069, 24251, 24269, 24740, 24753, 25632, 26295, 26357, 27104, 27752, 28098, 28100, 28404, 28660, 29179, 29512, 29801, 29909, 30085, 30149, 30337, 30523, 30601, 30776, 30780, 30877, 31922, 32220, 32318, 32691, 33979, 34313, 34459, 35289, 35395]
[35459, 35952, 35988, 36443, 37034, 37095, 37701, 38406, 38580, 38767, 38919, 39121, 39556, 39690, 40174, 40188, 40802, 40812, 41170, 41549, 41876, 41992, 41996, 42053, 42302, 42970, 43147, 43183, 43336, 43613, 43769, 43947, 44000, 44253, 44991, 45310, 45761, 45976, 46091, 46373, 46869, 46893, 47037, 47053, 48204, 48332, 

In [63]:
# 探索
cnt = 0
for _ in range(10000):
    x = randint(0, MAX)
    cnt += x in ss

print(cnt)

27
